In [1]:
import math

class DGIM:
    def __init__(self, window_size):
        self.window_size = window_size
        self.max_buckets = int(math.log2(window_size)) + 2
        self.buckets = {i: [] for i in range(self.max_buckets)}
        self.timestamp = 0

    def update(self, bit):
        if bit == 1:
            self.buckets[0].append(self.timestamp)
            self._merge_buckets()
        self.timestamp += 1
        self._remove_old_buckets()

    def _merge_buckets(self):
        for k in range(self.max_buckets):
            if len(self.buckets[k]) > 2:
                # Merge the oldest two buckets
                timestamp1 = self.buckets[k].pop(0)
                timestamp2 = self.buckets[k].pop(0)
                if k + 1 < self.max_buckets:
                    self.buckets[k + 1].append(timestamp1)
                else:
                    # Drop if beyond max_buckets (unlikely due to window_size)
                    pass

    def _remove_old_buckets(self):
        oldest_timestamp = self.timestamp - self.window_size
        for k in range(self.max_buckets):
            # Remove buckets that are outside the window
            self.buckets[k] = [ts for ts in self.buckets[k] if ts >= oldest_timestamp]

    def count_ones(self):
        count = 0
        for k in range(self.max_buckets):
            if self.buckets[k]:
                # Add 2^k for each bucket (except the last, which may be partially outside the window)
                count += (2 ** k) * len(self.buckets[k])
        # Adjust for the last bucket's possible partial coverage
        if self.buckets.get(self.max_buckets - 1):
            oldest_in_last = self.buckets[self.max_buckets - 1][0]
            if oldest_in_last <= self.timestamp - self.window_size:
                count -= (oldest_in_last - (self.timestamp - self.window_size)) * (2 ** (self.max_buckets - 1))
        return count

# Example Usage
if __name__ == "__main__":
    dgim = DGIM(window_size=32)  # Count 1s in the last 32 bits

    # Simulate a binary stream: 1011101010101010101010101010101...
    stream = [1,0,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]

    for bit in stream:
        dgim.update(bit)

    estimated_ones = dgim.count_ones()
    actual_ones = sum(stream[-32:])  # Last 32 bits

    print(f"Estimated number of 1s in last 32 bits: {estimated_ones}")
    print(f"Actual number of 1s in last 32 bits: {actual_ones}")

Estimated number of 1s in last 32 bits: 17
Actual number of 1s in last 32 bits: 17
